<a href="https://colab.research.google.com/github/hongjinkong/opensw/blob/main/1119/deepfakedetection_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/mystlee/dfdc_deepfake_challenge.git

Cloning into 'dfdc_deepfake_challenge'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 134 (delta 23), reused 4 (delta 4), pack-reused 65 (from 1)
Receiving objects: 100% (134/134), 69.20 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [ ]:
!pip install -U tensorflow
!pip install -U keras
!pip install efficientnet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.1 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
pip install --upgrade tensorflow

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import os

# 이미지 로딩 및 전처리 함수
def load_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=target_size)
        img = img_to_array(img) / 255.0  # 정규화
        images.append(img)
    return np.array(images)

# Google Drive에서 데이터 로드 및 준비
deepfake_folder = '/content/drive/MyDrive/deepfake/'  # 실제 경로로 변경
image_paths = [os.path.join(deepfake_folder, f) for f in os.listdir(deepfake_folder) if f.endswith('.jpg') or f.endswith('.png')]

# 이미지 데이터 로드
x_images = load_images(image_paths)

# 라벨 생성 (FAKE: 0, REAL: 1)
y_labels = []
for img_path in image_paths:
    if 'fake' in img_path.lower():
        y_labels.append(0)  # FAKE
    else:
        y_labels.append(1)  # REAL
y_labels = np.array(y_labels)

# 데이터셋 분할 (train, validation, test)
x_train, x_temp, y_train, y_temp = train_test_split(x_images, y_labels, test_size=0.4, random_state=42)  # 60% train
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

# Encoder 모델 정의
def build_encoder(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
    return Model(inputs, encoded, name="encoder")

# Classifier 모델 정의
def build_classifier(input_shape):
    encoder = build_encoder(input_shape)
    encoder_output = encoder.output
    flat = layers.Flatten()(encoder_output)
    dense = layers.Dense(64, activation='relu')(flat)
    output = layers.Dense(1, activation='sigmoid')(dense)  # 이진 분류
    classifier = Model(encoder.input, output, name="classifier")
    return classifier

# 분류기 모델 생성
input_shape = (128, 128, 3)
classifier = build_classifier(input_shape)

# 분류 모델 컴파일
classifier.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# 분류기 모델 학습
classifier.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Grad-CAM Heatmap 계산
def get_gradcam_heatmap(model, image, class_idx):
    # 모델의 마지막 Conv2D 레이어를 찾음
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer_name = layer.name
            break

    # 마지막 Conv2D 레이어의 출력 부분 모델과 이후의 분류기 부분 모델을 생성
    last_conv_layer_model = Model(inputs=model.inputs, outputs=model.get_layer(last_conv_layer_name).output)
    classifier_model = Model(inputs=model.get_layer(last_conv_layer_name).output, outputs=model.outputs)

    # 이미지를 텐서로 변환
    image = tf.convert_to_tensor(np.expand_dims(image, axis=0), dtype=tf.float32)
    with tf.GradientTape() as tape:
        # Conv 레이어 출력 및 예측값 계산
        conv_output = last_conv_layer_model(image)
        tape.watch(conv_output)
        preds = classifier_model(conv_output)
        class_output = preds[:, class_idx]

    # 그라디언트 계산
    grads = tape.gradient(class_output, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # heatmap 생성
    conv_output = conv_output[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)

    # heatmap 정규화
    heatmap = np.maximum(heatmap, 0)  # 음수 제거
    heatmap /= np.max(heatmap) + 1e-8  # 정규화

    # Heatmap이 비어있는지 확인
    if np.all(heatmap == 0):
        print("Warning: Generated heatmap is empty.")
    return heatmap  # numpy 배열로 반환


# Heatmap에서 Bounding Box 계산
def get_gradcam_bbox(heatmap, threshold=0.5):
    heatmap = np.uint8(255 * heatmap)
    threshold_value = threshold * 255
    coords = np.column_stack(np.where(heatmap >= threshold_value))
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    return x_min, y_min, x_max, y_max

# Grad-CAM + Bounding Box 시각화
def display_gradcam_with_bbox(model, image, predicted_class, label, threshold=0.5):
    # Grad-CAM heatmap 계산
    heatmap = get_gradcam_heatmap(model, image, predicted_class)

    if heatmap is None or np.all(heatmap == 0):
        print("No heatmap generated, skipping visualization.")
        return

    # Heatmap을 원본 이미지 크기로 조정
    heatmap_resized = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)  # heatmap의 값 0~255로 변환
    heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)  # 색상 맵 적용

    # 원본 이미지와 heatmap 합성
    superimposed_img = cv2.addWeighted(heatmap_color, 0.3, np.uint8(image * 255), 0.7, 0)  # 비율 0.3:0.7로 조정

    # Threshold를 적용하여 bounding box를 찾기
    ret, thresh = cv2.threshold(heatmap_resized, threshold * 255, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 각 contour에 대해 bounding box를 그리기
    for contour in contours:
        if cv2.contourArea(contour) > 100:  # 작은 영역은 무시
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(superimposed_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            # 딥페이크/리얼 텍스트 표시
            cv2.putText(superimposed_img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 결과 이미지 시각화
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()


# 테스트 이미지와 Grad-CAM 시각화
test_image = x_test[0]
predicted_class = int(classifier.predict(np.expand_dims(test_image, axis=0))[0, 0] > 0.5)
label = "Deepfake" if predicted_class == 0 else "Real"
display_gradcam_with_bbox(classifier, test_image, predicted_class, label, threshold=0.5)


Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - accuracy: 0.8617 - loss: 0.1279 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 2.7718e-42 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accura

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
No heatmap generated, skipping visualization.


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_108']. Received: the structure of inputs=*
  warnings.warn(


In [6]:
!setup.py install

/bin/bash: line 1: setup.py: command not found


In [11]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.utils import Sequence


# 이미지 로딩 및 전처리 함수
def load_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=target_size)
        img = img_to_array(img) / 255.0  # 정규화
        images.append(img)
    return np.array(images)

# Google Drive에서 데이터 로드 및 준비
deepfake_folder = '/content/drive/MyDrive/deepfake/'  # 실제 경로로 변경
image_paths = [os.path.join(deepfake_folder, f) for f in os.listdir(deepfake_folder) if f.endswith('.jpg') or f.endswith('.png')]

# 이미지 데이터 로드
x_images = load_images(image_paths)

# 라벨 생성 (FAKE: 0, REAL: 1)
y_labels = []
for img_path in image_paths:
    if 'fake' in img_path.lower():
        y_labels.append(0)  # FAKE
    else:
        y_labels.append(1)  # REAL
y_labels = np.array(y_labels)

# 데이터셋 분할 (train, validation, test)
x_train, x_temp, y_train, y_temp = train_test_split(x_images, y_labels, test_size=0.4, random_state=42)  # 60% train
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

# 데이터 증강 (augmentation)
datagen = ImageDataGenerator(
    rotation_range=15,        # 이미지 회전
    width_shift_range=0.1,    # 가로 이동
    height_shift_range=0.1,   # 세로 이동
    horizontal_flip=True      # 수평 뒤집기
)

# 훈련 데이터에 데이터 증강 적용
datagen.fit(x_train)

# Encoder 모델 정의
def build_encoder(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
    return Model(inputs, encoded, name="encoder")

# Classifier 모델 정의 (드롭아웃과 L2 정규화 추가)
def build_classifier_with_dropout(input_shape):
    encoder = build_encoder(input_shape)
    encoder_output = encoder.output
    flat = layers.Flatten()(encoder_output)
    dense = layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(flat)  # L2 정규화 추가
    dropout = layers.Dropout(0.5)(dense)  # 드롭아웃 추가
    output = layers.Dense(1, activation='sigmoid')(dropout)  # 이진 분류
    classifier = Model(encoder.input, output, name="classifier_with_dropout")
    return classifier

# 분류기 모델 생성 (드롭아웃 포함)
input_shape = (128, 128, 3)
classifier = build_classifier_with_dropout(input_shape)

# 분류 모델 컴파일
classifier.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
epochs = 50  # 에포크 수
classifier.fit(datagen.flow(x_train, y_train, batch_size=32),
              epochs=epochs,
              validation_data=(x_val, y_val), verbose=1)

# Grad-CAM 함수 수정
def get_gradcam_heatmap(model, image, class_idx):
    # 모델 구조에서 마지막 Conv2D 레이어를 자동으로 탐색
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if isinstance(layer, layers.Conv2D):
            last_conv_layer_name = layer.name
            break

    if not last_conv_layer_name:
        raise ValueError("No Conv2D layer found in the model.")

    # 마지막 Conv2D 레이어를 기준으로 모델을 다시 정의
    last_conv_layer_model = Model(inputs=model.inputs, outputs=model.get_layer(last_conv_layer_name).output)
    classifier_model = Model(inputs=model.get_layer(last_conv_layer_name).output, outputs=model.outputs)

    # 이미지 전처리
    image_input = np.expand_dims(image, axis=0)  # 배치 차원 추가
    image_input = tf.convert_to_tensor(image_input, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(image_input)
        conv_output = last_conv_layer_model(image_input)
        preds = classifier_model(conv_output)
        class_output = preds[:, class_idx]

    # 그라디언트 계산
    grads = tape.gradient(class_output, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))  # 채널 차원에서 평균

    # Heatmap 생성
    conv_output = conv_output[0]  # (height, width, channels) 형태로 변환
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)  # 음수 제거
    heatmap /= np.max(heatmap) + 1e-8  # 정규화

    return heatmap  # 생성된 Heatmap 반환

# Grad-CAM + Bounding Box 시각화
def display_gradcam_with_bbox(model, image, predicted_class, label, threshold=0.5):
    # Grad-CAM heatmap 계산
    heatmap = get_gradcam_heatmap(model, image, predicted_class)

    if heatmap is None or np.all(heatmap == 0):
        print("No heatmap generated, skipping visualization.")
        return

    # Heatmap을 원본 이미지 크기로 조정
    heatmap_resized = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)  # heatmap의 값 0~255로 변환
    heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)  # 색상 맵 적용

    # 원본 이미지와 heatmap 합성
    superimposed_img = cv2.addWeighted(heatmap_color, 0.3, np.uint8(image * 255), 0.7, 0)  # 비율 0.3:0.7로 조정

    # Threshold를 적용하여 bounding box를 찾기
    ret, thresh = cv2.threshold(heatmap_resized, threshold * 255, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 각 contour에 대해 bounding box를 그리기
    for contour in contours:
        if cv2.contourArea(contour) > 100:  # 작은 영역은 무시
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(superimposed_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            # 딥페이크/리얼 텍스트 표시
            cv2.putText(superimposed_img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 결과 이미지 시각화
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# 테스트 이미지와 Grad-CAM 시각화
test_image = x_test[0]  # 이미 (128, 128, 3) 형태임
predicted_class = int(classifier.predict(np.expand_dims(test_image, axis=0))[0, 0] > 0.5)  # 예측을 위해서는 배치 차원이 필요합니다.
label = "Deepfake" if predicted_class == 0 else "Real"
display_gradcam_with_bbox(classifier, test_image, predicted_class, label, threshold=0.5)


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 212ms/step - accuracy: 0.8477 - loss: 0.2330 - val_accuracy: 1.0000 - val_loss: 0.0688
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 1.0000 - loss: 0.0633 - val_accuracy: 1.0000 - val_loss: 0.0474
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 193ms/step - accuracy: 1.0000 - loss: 0.0434 - val_accuracy: 1.0000 - val_loss: 0.0331
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 1.0000 - loss: 0.0305 - val_accuracy: 1.0000 - val_loss: 0.0239
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 1.0000 - loss: 0.0222 - val_accuracy: 1.0000 - val_loss: 0.0177
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - accuracy: 1.0000 - loss: 0.0165 - val_accuracy: 1.0000 - val_loss: 0.0134
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - accuracy: 1.0000 - loss: 0.0126 - val_accuracy: 1.0000 - val_loss: 0.0103
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 1.0000 - loss: 0.0097 - val_accuracy: 1.

/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_65']. Received: the structure of inputs=*
  warnings.warn(
